In [59]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import unicodedata
import pythainlp
import re
import emoji
import tensorflow as tf
import numpy as np
from sentence_transformers import SentenceTransformer, util
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [4]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word)]

    res = ungroup_emoji(res)
    return res

In [5]:
train_df = pd.read_csv('./train_data_fake_news.csv')
val_df = pd.read_csv('./valid_data_fake_news.csv')
test_df = pd.read_csv('./test_data_fake_news.csv')

In [6]:
train_df

,text,label
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1
...,...,...
2358,เดี๋ยวนี้มีใบทุเรียนน้ำแกโรคมะเร็งหายขาดจ้า,1
2359,อาหาร8ประเภทที่ไม่ควรนำไปอุ่นซ้ำอีกรอบ|||จะต้อ...,1
2360,เภสัชเผยยากลุ่มNSAIDsห้ามกินร่วมกันเสี่ยงอันตร...,0
2361,สบส.ฟันธง!!วัยทำงานเกือบครึ่งอนาคตเสี่ยงโรคเรื...,0


In [35]:
train_df_pre = train_df.copy()
val_df_pre = val_df.copy()
test_df_pre = test_df.copy()
train_df_pre['text'] = train_df_pre['text'].map(preprocess_func)
val_df_pre['text'] = val_df_pre['text'].map(preprocess_func)
test_df_pre['text'] = test_df_pre['text'].map(preprocess_func)

In [36]:
word_count = []
for text in train_df_pre['text']:
    for word in text:
        word_count.append(word)

In [37]:
VOCAB_SIZE = len(set(word_count))
VOCAB_SIZE

25273

In [38]:
MAX_SEQ_LEN = 250

In [39]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(train_df_pre['text'])

In [41]:
train_seq = tokenizer.texts_to_sequences(train_df_pre['text'])

In [43]:
train_seq[0]

[1192, 3278, 15475, 40, 24, 4, 41, 83, 904, 540]

In [44]:
train_pad = pad_sequences(train_seq, maxlen=MAX_SEQ_LEN)

In [45]:
train_pad

array([[   0,    0,    0, ...,   83,  904,  540],
       [   0,    0,    0, ..., 1066, 2257,  619],
       [  48,   29,  256, ...,  763,   33, 2381],
       ...,
       [  50,   98,   51, ...,  171,   73,   26],
       [   3,  232,    7, ...,    5,   35, 2323],
       [   0,    0,    0, ..., 3577, 3810,   73]])

In [48]:
val_seq = tokenizer.texts_to_sequences(val_df_pre['text'])
val_pad = pad_sequences(val_seq, maxlen=MAX_SEQ_LEN)

In [47]:
print(train_df_pre['text'][0])
print(train_seq[0])

['ไก่', 'ลุง', 'เคน', 'กิน', 'แล้ว', 'มี', 'แต่', 'สาร', 'เร่ง', 'โต']
[1192, 3278, 15475, 40, 24, 4, 41, 83, 904, 540]
